In [1]:
from search import *
from agents import*

In [2]:
class Tpiece(Thing): #Create a movable piece class
    pass

class Board(Environment): #Create an environment like last assignment
    pass

In [3]:
class PSA(SimpleProblemSolvingAgentProgram):
    def __call__(self, board): #AIMA overrided function because it had some things wrong
        self.percept = board.things #percept will just be the list of objects in the board environment
        self.state = self.update_state(self.percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq.append(self.search(problem))
            #This was originally self.seq = blabla in AIMA which obviously gives an error when trying to pop it later wtf AIMA
            if not self.seq:
                return None
        return self.seq.pop(0) #self.seq goes back empty for working the next goal
    
    def update_state(self, percept): #from board.things it builds a tuple of tuples to represent the initial state of the current step
        #Format: (('Identification char', (location)), next object of the percept in the same format)
        #Example: ( ('A', (1, 1)), ('T', (5, 2)) ) <- means a state with agent in 1, 1 and movable piece in 5, 2
        #Output needs to be tuple because the AIMA graph search function needs hashable data structures to add to the explored set
        
        state = [] #Easier to build the state as a list
        for p in percept:
            if isinstance(p, Agent):
                state.append(('A', p.location))
            elif isinstance(p, Tpiece):
                state.append(('T', p.location))
        state = tuple(state) #convert the state to tuple
        return state

    def formulate_goal(self, state): #returns the desired state of the current step
        temp = dict(state)
        #converting the state temporarily to a dictionary makes it easier to manip the data, example: {'A': (1,1), 'T': (5, 5)}
        
        if temp['A'][0] == temp['T'][0] and temp['A'][1] == temp['T'][1] - 1: #If the agent is already to the left of the movable piece
            temp['A'] = (5, 4)
            temp['T'] = (5, 5)
            #The desired state would be to finish the job and have the piece be in 5, 5
        else: #The agent is not to the left of the piece yet
            temp['A'] = (temp['T'][0], temp['T'][1] - 1)
            #The desired state is to initially position the agent to the left of the piece

        #This next block simply builds a proper tuple formatted state from the edited dictionary
        goal = []
        for k in temp.items():
            goal.append(k)
        goal = tuple(goal)
        return goal
        '''TODO: obviously develop this function so it can formulate proper goals in all boards. Board 1 only requires "2 goals"
        which is first to approach the piece and then push it, but some boards will require previous goals such as clearing
        a path from obstacles'''

    def formulate_problem(self, state, goal): #create a problem object with the initial state and goal state for this step
        problem = PushnPull(state, goal)
        return problem
    
    def search(self, problem): #Uses some search algorithm included in AIMA to solve the problem
        return graph_search(problem, [])
        

In [4]:
class PushnPull(Problem):
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        actions = []
        temp = dict(state)
        #converting the state temporarily to a dictionary makes it easier to manip the data, example: {'A': (1,1), 'T': (5, 5)}
        
        if temp['A'][0] == temp['T'][0] and temp['A'][1] == temp['T'][1] - 1: #if agent is to the left of the piece
            actions.append('pushRight')
        
        else: #Agent still far from the piece
            if temp['A'][1] == 1: #moving left is out of the grid so its illegal 
                actions.append('moveRight')
            elif temp['A'][1] == 6: #moving right is out of the grid so its illegal
                actions.append('moveLeft')
            else: #any horizontal move action is legal
                actions.append('moveRight')
                actions.append('moveLeft')
            
            if temp['A'][0] == 1: #moving up is out of the grid so its illegal
                actions.append('moveDown')
            elif temp['A'][0] == 6: #moving down is out of the grid so its illegal
                actions.append('moveUp')
            else: #any vetical move action is legal
                actions.append('moveDown')
                actions.append('moveUp')
            
        return actions
        '''TODO: obviously implement more actions for the other scenarios and goals that arise in different boards'''
    
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        temp = dict(state)
        #converting the state temporarily to a dictionary makes it easier to manip the data, example: {'A': (1,1), 'T': (5, 5)}
        
        if action == 'pushRight':
            temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            temp['T'] = (temp['T'][0], temp['T'][1] + 1)
            #The new state has both the piece and the agent one cell to the right
            
        else:
            if action == 'moveRight':
                temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            elif action == 'moveLeft':
                temp['A'] = (temp['A'][0], temp['A'][1] - 1)
            elif action == 'moveUp':
                temp['A'] = (temp['A'][0] - 1, temp['A'][1])
            elif action == 'moveDown':
                temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            #The new state has the agent move accordingly and the piece position is unchanged
        
        #This next block simply builds a proper tuple formatted state from the edited dictionary
        result = []
        for k in temp.items():
            result.append(k)
        result = tuple(result)
        return result
        '''TODO: obviously handle the proper state changes that arise from other actions and scenarios not implemented yet in
        self.actions()'''
    
    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if state == self.goal:
            return True
        else:
            return False

In [5]:
def graph_search(problem, frontier): #This is just a default AIMA search algorithm which is ran from an agent's search method
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

In [6]:
#This is the process for using the code above and solving board 1 of the assignment

#Create an empty board, a movable piece and an agent
board1 = Board()
t1 = Tpiece()
mike = PSA()

#Add things to the board/env as specified for board1 in the assignment
board1.add_thing(mike, (1, 1))
board1.add_thing(t1, (5, 2))

#Reach the first goal state (agent wants to go near the piece to move it) and save the resulting node class object to a variable
first_goal = mike(board1) #This executes the __call__ method of the agent which is where the whole process starts

print("Sequence of states to get to the left side of the movable piece:")
print(str(first_goal.path()) + "\n") #each node has stored the sequence of nodes used to get to that node
print("Sequence of actions to get to the left side of the movable piece:")
print(str(first_goal.solution()) + "\n") #each node has stored the sequence of actions to get to that node

#Now we need to achieve the second goal which is to push the piece to the target
#First lets update the environment/board (this could be done from a fucntion inside the agent or somewhere else, putting it here provisionally)
board1.things = [] #clean the old version of the env
temp = dict(first_goal.path()[len(first_goal.path()) - 1].state) #get the last state reached (the previous goal state)
for k in temp.items(): #Use the previous goal state to update the env/board
    if k[0] == 'A':
        board1.add_thing(mike, k[1]) #agent is now in 5, 1 to the left of the piece
    elif k[0] == 'T':
        board1.add_thing(t1, k[1]) #piece actually hasn't moved yet from 5, 2 but its checked again regardless
#TODO: graphical representation of the old board and the new one after each step (after each goal state is reached)

#Reach the second goal state (piece is in target and agent is 1 cell to the left of target) and save the node class object to a variable
second_goal = mike(board1)

print("Sequence of states to push the movable piece to the target")
print(str(second_goal.path()) + "\n")
print("Sequence of actions to push the movable piece to the target")
print(str(second_goal.solution()) + "\n")

print("Board 1 solved!")
#Obviously with more complex boards, more steps will be needed (more "subgoals"), here there were only 2 subgoals

Sequence of states to get to the left side of the movable piece:
[<Node (('A', (1, 1)), ('T', (5, 2)))>, <Node (('A', (2, 1)), ('T', (5, 2)))>, <Node (('A', (3, 1)), ('T', (5, 2)))>, <Node (('A', (4, 1)), ('T', (5, 2)))>, <Node (('A', (5, 1)), ('T', (5, 2)))>]

Sequence of actions to get to the left side of the movable piece:
['moveDown', 'moveDown', 'moveDown', 'moveDown']

Sequence of states to push the movable piece to the target
[<Node (('A', (5, 1)), ('T', (5, 2)))>, <Node (('A', (5, 2)), ('T', (5, 3)))>, <Node (('A', (5, 3)), ('T', (5, 4)))>, <Node (('A', (5, 4)), ('T', (5, 5)))>]

Sequence of actions to push the movable piece to the target
['pushRight', 'pushRight', 'pushRight']

Board 1 solved!
